In [252]:
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import utils
import os
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [226]:
data_dir = '../data/fma_metadata/'
echonest = utils.load(data_dir + 'echonest.csv')
# Load metadata and features.
tracks = utils.load(data_dir + 'tracks.csv')
genres = utils.load(data_dir + 'genres.csv')
features = utils.load(data_dir + 'features.csv')
echonest = utils.load(data_dir + 'echonest.csv')

In [227]:
echonest.head()

echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                                                        \
                                           metadata                      
         speechiness    tempo   valence  album_date         album_name   
track_id                                                                 
2           0.159310  165.922  0.576661         NaN                NaN   
3           0.461818  126.957  0.269240         NaN                NaN   
5           0.124595  100.260  0.621661         NaN                NaN   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker   
134         0.525519  114.290  0.894072         NaN                NaN   

             ...                                                       \
             ...     temporal_features                                  
             ...                   214        215       216       217   
track_id     ...                                                        
2            ...             -1.992303   6.805694  0.233070  0.192880   
3            ...             -1.582331   8.889308  0.258464  0.220905   
5            ...             -2.288358  11.527109  0.256821  0.237820   
10           ...             -3.662988  21.508228  0.283352  0.267070   
134          ...             -1.452696   2.356398  0.234686  0.199550   

                                                                        
                                                                        
               218      219       220       221        222         223  
track_id                                                                
2         0.027455  0.06408   3.67696   3.61288  13.316690  262.929749  
3         0.081368  0.06413   6.08277   6.01864  16.673548  325.581085  
5         0.060122  0.06014   5.92649   5.86635  16.013849  356.755737  
10        0.125704  0.08082   8.41401   8.33319  21.317064  483.403809  
134       0.149332  0.06440  11.26707  11.20267  26.454180  751.147705  

[5 rows x 249 columns]

In [228]:
tracks.columns

MultiIndex(levels=[['album', 'artist', 'set', 'track'], ['active_year_begin', 'active_year_end', 'associated_labels', 'bio', 'bit_rate', 'comments', 'composer', 'date_created', 'date_recorded', 'date_released', 'duration', 'engineer', 'favorites', 'genre_top', 'genres', 'genres_all', 'id', 'information', 'interest', 'language_code', 'latitude', 'license', 'listens', 'location', 'longitude', 'lyricist', 'members', 'name', 'number', 'producer', 'publisher', 'related_projects', 'split', 'subset', 'tags', 'title', 'tracks', 'type', 'website', 'wikipedia_page']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [5, 7, 9, 11, 12, 16, 17, 22, 29, 34, 35, 36, 37, 0, 1, 2, 3, 5, 7, 12, 16, 20, 23, 24, 26, 27, 31, 34, 38, 39, 32, 33, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 17, 18, 19, 21, 22, 25, 28, 30, 34, 35]])

In [229]:
indexed_genres = pd.DataFrame(tracks['track']['genre_top'])
indexed_genres['genre_top'].unique()

[Hip-Hop, Pop, NaN, Rock, Experimental, ..., Country, Classical, Old-Time / Historic, Instrumental, Easy Listening]
Length: 17
Categories (16, object): [Hip-Hop, Pop, Rock, Experimental, ..., Classical, Old-Time / Historic, Instrumental, Easy Listening]

In [230]:
echonest['echonest'].head(5)

audio_features                                                    \
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                           metadata                     \
         speechiness    tempo   valence  album_date         album_name   
track_id                                                                 
2           0.159310  165.922  0.576661         NaN                NaN   
3           0.461818  126.957  0.269240         NaN                NaN   
5           0.124595  100.260  0.621661         NaN                NaN   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker   
134         0.525519  114.290  0.894072         NaN                NaN   

             ...     temporal_features                                 \
             ...                   214        215       216       217   
track_id     ...                                                        
2            ...             -1.992303   6.805694  0.233070  0.192880   
3            ...             -1.582331   8.889308  0.258464  0.220905   
5            ...             -2.288358  11.527109  0.256821  0.237820   
10           ...             -3.662988  21.508228  0.283352  0.267070   
134          ...             -1.452696   2.356398  0.234686  0.199550   

                                                                        
               218      219       220       221        222         223  
track_id                                                                
2         0.027455  0.06408   3.67696   3.61288  13.316690  262.929749  
3         0.081368  0.06413   6.08277   6.01864  16.673548  325.581085  
5         0.060122  0.06014   5.92649   5.86635  16.013849  356.755737  
10        0.125704  0.08082   8.41401   8.33319  21.317064  483.403809  
134       0.149332  0.06440  11.26707  11.20267  26.454180  751.147705  

[5 rows x 249 columns]

In [231]:
echonest.columns[0:2]

MultiIndex(levels=[['echonest'], ['audio_features', 'metadata', 'ranks', 'social_features', 'temporal_features'], ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '12

In [232]:
df = echonest['echonest'].join(indexed_genres, how='left')

/Users/claytonblythe/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [233]:
df.head()

,"(audio_features, acousticness)","(audio_features, danceability)","(audio_features, energy)","(audio_features, instrumentalness)","(audio_features, liveness)","(audio_features, speechiness)","(audio_features, tempo)","(audio_features, valence)","(metadata, album_date)","(metadata, album_name)",...,"(temporal_features, 215)","(temporal_features, 216)","(temporal_features, 217)","(temporal_features, 218)","(temporal_features, 219)","(temporal_features, 220)","(temporal_features, 221)","(temporal_features, 222)","(temporal_features, 223)",genre_top
track_id,,,,,,,,,,,,,,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,NaN,NaN,...,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749,Hip-Hop
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,NaN,NaN,...,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085,Hip-Hop
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,NaN,NaN,...,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737,Hip-Hop
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008-03-11,Constant Hitmaker,...,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809,Pop
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,NaN,NaN,...,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705,Hip-Hop


In [234]:
df['genre_top'].value_counts()

Rock                   3892
Electronic             2170
Hip-Hop                 910
Folk                    874
Old-Time / Historic     357
Pop                     346
Classical               265
Jazz                    241
International           133
Instrumental             84
Blues                    66
Experimental             17
Spoken                    0
Soul-RnB                  0
Easy Listening            0
Country                   0
Name: genre_top, dtype: int64

In [235]:
df[df.columns[0:25]].head(5)

,"(audio_features, acousticness)","(audio_features, danceability)","(audio_features, energy)","(audio_features, instrumentalness)","(audio_features, liveness)","(audio_features, speechiness)","(audio_features, tempo)","(audio_features, valence)","(metadata, album_date)","(metadata, album_name)",...,"(ranks, artist_discovery_rank)","(ranks, artist_familiarity_rank)","(ranks, artist_hotttnesss_rank)","(ranks, song_currency_rank)","(ranks, song_hotttnesss_rank)","(social_features, artist_discovery)","(social_features, artist_familiarity)","(social_features, artist_hotttnesss)","(social_features, song_currency)","(social_features, song_hotttnesss)"
track_id,,,,,,,,,,,,,,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.388990,0.386740,0.406370,0.000000,0.000000
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.388990,0.386740,0.406370,0.000000,0.000000
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.388990,0.386740,0.406370,0.000000,0.000000
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008-03-11,Constant Hitmaker,...,2635.0,2544.0,397.0,115691.0,67609.0,0.557339,0.614272,0.798387,0.005158,0.354516
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.388990,0.386740,0.406370,0.000000,0.000000


In [236]:
audio_features_df = df[df.columns[0:8]]
audio_features_df.head()

,"(audio_features, acousticness)","(audio_features, danceability)","(audio_features, energy)","(audio_features, instrumentalness)","(audio_features, liveness)","(audio_features, speechiness)","(audio_features, tempo)","(audio_features, valence)"
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [237]:
latitude = pd.DataFrame(df[df.columns[10]])
longitude = pd.DataFrame(df[df.columns[12]])

In [238]:
my_df = audio_features_df.join(latitude).join(longitude).join(temporal_df)

/Users/claytonblythe/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [239]:
my_df.head()

,"(audio_features, acousticness)","(audio_features, danceability)","(audio_features, energy)","(audio_features, instrumentalness)","(audio_features, liveness)","(audio_features, speechiness)","(audio_features, tempo)","(audio_features, valence)","(metadata, artist_latitude)","(metadata, artist_longitude)",...,"(temporal_features, 214)","(temporal_features, 215)","(temporal_features, 216)","(temporal_features, 217)","(temporal_features, 218)","(temporal_features, 219)","(temporal_features, 220)","(temporal_features, 221)","(temporal_features, 222)","(temporal_features, 223)"
track_id,,,,,,,,,,,,,,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,32.6783,-83.2230,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,32.6783,-83.2230,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,32.6783,-83.2230,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,39.9523,-75.1624,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,32.6783,-83.2230,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705


In [240]:
my_df.describe()[0:10]

,"(audio_features, acousticness)","(audio_features, danceability)","(audio_features, energy)","(audio_features, instrumentalness)","(audio_features, liveness)","(audio_features, speechiness)","(audio_features, tempo)","(audio_features, valence)","(metadata, artist_latitude)","(metadata, artist_longitude)",...,"(temporal_features, 214)","(temporal_features, 215)","(temporal_features, 216)","(temporal_features, 217)","(temporal_features, 218)","(temporal_features, 219)","(temporal_features, 220)","(temporal_features, 221)","(temporal_features, 222)","(temporal_features, 223)"
count,1.312900e+04,13129.000000,13129.000000,13129.000000,13129.000000,13129.000000,13129.000000,13129.000000,9770.000000,9770.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,5.246876e-01,0.487290,0.537516,0.640536,0.187804,0.099174,123.080061,0.439761,39.961892,-49.984040,...,-1.931167,7.883920,0.287116,0.247037,0.121842,0.066946,7.356944,7.289997,16.641947,376.613388
std,3.837186e-01,0.190148,0.278049,0.361430,0.158051,0.137381,35.015137,0.276028,15.554275,59.377974,...,1.066580,7.399431,0.067123,0.040736,0.110616,0.007159,3.354948,3.352938,6.871484,219.555551
min,9.035000e-07,0.051307,0.000020,0.000000,0.025297,0.022324,12.753000,0.000010,-45.867100,-155.434000,...,-3.662988,-0.260981,0.233070,0.192880,0.025242,0.060140,2.436690,2.372650,3.897095,37.866043
25%,1.037726e-01,0.344759,0.321300,0.323466,0.101406,0.036932,95.967000,0.197321,37.784827,-87.632400,...,-2.485935,1.725808,0.251288,0.215566,0.051956,0.063687,5.364107,5.302982,13.440321,255.445698
50%,5.739848e-01,0.485635,0.549113,0.838134,0.119002,0.049019,120.057000,0.417743,40.781400,-74.007100,...,-1.787317,7.847501,0.264633,0.252445,0.103536,0.064105,7.248390,7.175915,16.343699,341.168411
75%,9.207270e-01,0.629094,0.776254,0.918244,0.211041,0.085452,145.318000,0.665575,47.254575,-0.127140,...,-1.323196,11.748224,0.293072,0.274020,0.139893,0.067133,10.194250,10.122095,21.483203,503.110214
max,9.957965e-01,0.968645,0.999964,0.998016,0.980330,0.966177,251.072000,0.999990,68.927500,174.770000,...,-0.457298,21.508228,0.437503,0.307390,0.368764,0.080820,11.267070,11.202670,26.454180,751.147705


In [241]:
my_df.replace('', np.nan, inplace=True);
my_df.dropna()

,"(audio_features, acousticness)","(audio_features, danceability)","(audio_features, energy)","(audio_features, instrumentalness)","(audio_features, liveness)","(audio_features, speechiness)","(audio_features, tempo)","(audio_features, valence)","(metadata, artist_latitude)","(metadata, artist_longitude)",...,"(temporal_features, 214)","(temporal_features, 215)","(temporal_features, 216)","(temporal_features, 217)","(temporal_features, 218)","(temporal_features, 219)","(temporal_features, 220)","(temporal_features, 221)","(temporal_features, 222)","(temporal_features, 223)"
track_id,,,,,,,,,,,,,,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,32.6783,-83.2230,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,32.6783,-83.2230,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,32.6783,-83.2230,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,39.9523,-75.1624,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,32.6783,-83.2230,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705
139,0.106550,0.260911,0.607067,0.835087,0.223676,0.030569,196.961,0.160267,41.8239,-71.4120,...,-3.078667,12.411567,0.270802,0.272700,0.025242,0.06404,2.43669,2.37265,3.897095,37.866043
140,0.376312,0.734079,0.265685,0.669581,0.085995,0.039068,107.952,0.609991,41.8239,-71.4120,...,-0.934696,-0.260981,0.322232,0.277980,0.136747,0.07533,9.86272,9.78739,21.981621,562.229431
141,0.963657,0.435933,0.075632,0.345493,0.105686,0.026658,33.477,0.163950,41.8239,-71.4120,...,-0.457298,-0.165962,0.437503,0.307390,0.368764,0.06263,11.18884,11.12621,13.481531,232.993546


In [242]:
my_df.shape

(13129, 234)

In [243]:
# Mathematical representation of one song by it's temporal features
numpy_df = my_df.as_matrix()
numpy_df.shape

(13129, 234)

In [244]:
numpy_df[0][0:10]

array([  4.16675233e-01,   6.75893985e-01,   6.34476268e-01,
         1.06280683e-02,   1.77646571e-01,   1.59310065e-01,
         1.65922000e+02,   5.76660988e-01,   3.26783000e+01,
        -8.32230000e+01])

In [245]:
indexes = my_df.index
indexes

Int64Index([     2,      3,      5,     10,    134,    139,    140,    141,
               142,    144,
            ...
            124852, 124853, 124854, 124855, 124856, 124857, 124862, 124863,
            124864, 124911],
           dtype='int64', name='track_id', length=13129)

In [248]:
genres = df['genre_top']
genres.value_counts()

Rock                   3892
Electronic             2170
Hip-Hop                 910
Folk                    874
Old-Time / Historic     357
Pop                     346
Classical               265
Jazz                    241
International           133
Instrumental             84
Blues                    66
Experimental             17
Spoken                    0
Soul-RnB                  0
Easy Listening            0
Country                   0
Name: genre_top, dtype: int64

In [261]:
my_df = my_df.dropna()
numpy_df = 

In [263]:
similarities = cosine_similarity(numpy_df)
print('pairwise dense output:\n {}\n'.format(similarities))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').